# Trabalho 1 - MC853B

**Alunos:** 

Lucca Miranda Nunes - XXXX

Luiz Felipe Corradini Rego Costa - 230613

Pablo Henrique Almeida Mendes - XXXX




**Doença de notificação compulsória escolhida:** Esquistossomose

## 1) Upload e investigação dos dados

### 1.1) Análise Exploratória dos dados

Os dados escolhidos para treinamento foram os da doença esquistossomose dos anos de 2021, 2022 e 2023. Os anos de 2024 em diante não estavam disponibilizados na base do SINAN. Todos os dados estão no diretório source/csv.

In [6]:
import pandas as pd

esq_21 = pd.read_csv('source/csv/ESQUBR21.csv')
esq_22 = pd.read_csv('source/csv/ESQUBR22.csv')
esq_23 = pd.read_csv('source/csv/ESQUBR23.csv')

In [7]:
print(esq_21.shape)
print(esq_22.shape)
print(esq_23.shape)

(2395, 54)
(2640, 54)
(3059, 54)


Com os três anos selecionados, obtemos, em primeiro momento, um total de 8094 instâncias. 

Precisamos investigar a quantidade de atributos-alvo em cada ano, a fim de garantir balanceamento nas classes dentro dos datasets de treino e de teste:

Valores correspondentes a cada desfecho possível do paciente:


1 - Cura

2 - Não Cura

3 - Óbito por 
esquistossomose 

4 -  Óbito por outras 
causas 

9 - Ignorado

In [29]:
esq_21['EVOLUCAO'].value_counts()

1.0    1205
9.0     282
3.0      69
2.0      65
4.0      25
Name: EVOLUCAO, dtype: int64

In [30]:
esq_22['EVOLUCAO'].value_counts()

1.0    1337
9.0     304
2.0      64
3.0      62
4.0      37
Name: EVOLUCAO, dtype: int64

In [31]:
esq_23['EVOLUCAO'].value_counts()

1.0    1413
9.0     283
3.0      61
2.0      47
4.0      30
Name: EVOLUCAO, dtype: int64

Como iremos usar os dados dos três anos, faz mais sentido concatená-los em um único DataFrame, a fim de realizar a análise em um único comando

In [33]:
esq_total = pd.concat([esq_21, esq_22, esq_23])
esq_total['EVOLUCAO'].value_counts()

1.0    3955
9.0     869
3.0     192
2.0     176
4.0      92
Name: EVOLUCAO, dtype: int64

### 1.2) Divisão do conjunto de treino e teste

O primeiro passo vai ser separar um conjunto de teste nos dados, já que esse conjunto não deve ser pré-processado como os de treinamento. Para garantir que 

In [ ]:
# Avaliar como vai ser a divisão dos dados, e avaliar também como garantir 10% de representatividade de cada classe nos conjuntos de treino e teste

## 2) Pré-processamento dos dados

Como o problema que iremos prever é de natureza binária (óbito ou não-óbito), precisamos remover todas as instâncias que a resposta foi "Ignorado" ou "Não-Cura". Se o caso foi ignorado, não podemos inserí-lo em nenhuma das duas classes, e a classificação de "Não-Cura" também não torna possível avaliar se o paciente veio a óbito ou não.

Podemos ver que, pelo número elevado de colunas, será necessário realizar uma análise quanto às colunas necessárias, para posteriormente avaliar se o total de instâncias apresentadas nos três anos serão suficientes para treinar o modelo.

#### 2.1) Removendo Colunas desnecessárias

O primeiro passo para pré-processar será **remover colunas desnecessárias do dataset**. 

Depois de uma primeira análise, a equipe selecionou 11 colunas que julgamos como mais relevantes para o treinamento do modelo:


*colocar info das colunas*


In [11]:
# consertar, ta errado
colunas_selecionadas = [
    'ID_UNIDADE', 'DT_SIN_PRI', 'CS_GESTANT',
    'ID_RG_RESI', 'AN_QUANT', 'OUTROS',
    'TRATAM', 'STCURA1', 'FORMA',
    'COUFINF', 'EVOLUCAO'
]

df_filtrado = esq_21[colunas_selecionadas]

df_filtrado.head()

,ID_UNIDADE,DT_SIN_PRI,CS_GESTANT,ID_RG_RESI,AN_QUANT,OUTROS,TRATAM,STCURA1,FORMA,COUFINF,EVOLUCAO
0,2267810,2020-12-10,6,NaN,0.0,NaN,9.0,NaN,NaN,33.0,9.0
1,2141078,2021-01-02,6,1468.0,1.0,3.0,1.0,1.0,1.0,31.0,1.0
2,9407375,2020-12-20,6,1465.0,1.0,1.0,1.0,NaN,1.0,31.0,1.0
3,3975827,2021-01-04,6,1435.0,1.0,3.0,1.0,NaN,NaN,NaN,1.0
4,9407375,2020-12-02,6,1465.0,1.0,1.0,1.0,NaN,1.0,31.0,1.0
